# Transfer files from HDFS to local

This notebook is used to transfer files from HDFS to local. This is needed because the files preprocessed in spark are stored in HDFS, while the last part of the preprocessing, done using pandas, takes place locally.

## Launching spark

In [1]:
%%configure
{"conf": {
    "spark.app.name": "lgptguys_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8983,application_1589299642358_3520,pyspark,idle,Link,Link,
8996,application_1589299642358_3534,pyspark,idle,Link,Link,
9028,application_1589299642358_3573,pyspark,idle,Link,Link,
9035,application_1589299642358_3582,pyspark,idle,Link,Link,
9054,application_1589299642358_3599,pyspark,idle,Link,Link,
9055,application_1589299642358_3601,pyspark,idle,Link,Link,
9059,application_1589299642358_3605,pyspark,idle,Link,Link,
9065,application_1589299642358_3611,pyspark,idle,Link,Link,
9070,application_1589299642358_3618,pyspark,idle,Link,Link,
9072,application_1589299642358_3621,pyspark,idle,Link,Link,


### Start Spark

In [2]:
# Initialization
%%spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9106,application_1589299642358_3668,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
unknown magic command '%spark'
UnknownMagic: unknown magic command '%spark'



### Transfer and save tables from hdfs to local 

Here we describe the process of loading a table on hdfs, saving it to a proper place so that we can load it in local and then save it.

First, we load the data in the HDFS folder of the user to make it available. For this we need to transfer the username of the user to spark

In [3]:
import os

username = os.environ['JUPYTERHUB_USER']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'JUPYTERHUB_USER'
Traceback (most recent call last):
  File "/usr/lib64/python2.7/UserDict.py", line 23, in __getitem__
    raise KeyError(key)
KeyError: 'JUPYTERHUB_USER'



In [4]:
%%send_to_spark -i username -t str -n username 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
unknown magic command '%send_to_spark'
UnknownMagic: unknown magic command '%send_to_spark'



In [5]:
my_files = ['stop_times_curated.csv',
            'stops_15km.csv', 'transfers.csv', 'stop_times_final_cyril.csv', "transfers_cyril.csv"]

for file in my_files:
    this_file = spark.read.csv('data/lgpt_guys/{}'.format(file), \
                                  header = True)    
    this_file.write.csv("/user/{0}/{1}".format(username, file.replace('.csv','')), \
                                 header = True, mode = 'overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'username' is not defined
Traceback (most recent call last):
NameError: name 'username' is not defined



In [6]:
%local

from hdfs3 import HDFileSystem
import pandas as pd
import numpy as np 

print(os.getcwd())

hdfs = HDFileSystem(host='hdfs://iccluster044.iccluster.epfl.ch', port=8020, user='ebouille')


my_folders = ['stop_times_curated', 
              'stops_15km', 'transfers', 'stop_times_final_cyril', "transfers_cyril"]

for folder in my_folders:
    print(folder)
    array_files = hdfs.glob('/user/{0}/{1}/*.csv'.format(username, folder))
    array = pd.DataFrame()
    for file in array_files:
        with hdfs.open(file) as f:
            array = array.append(pd.read_csv(f))

    array.to_csv('../data/{}.csv'.format(folder), header=True)

NameError: name 'os' is not defined